In [313]:
from sympy.interactive import printing
printing.init_printing(use_latex = True)
import numpy as np
# import sympy as sp
import os
import sys
sys.path.append(f"C:\\Users\\maurovlachus\\OneDrive - Technical University of Cluj-Napoca\\utcn\\master\\caa\\proiect\\p1_opamps\\ekvParameterExtraction\\ekvExtractionScripts")
from siUnits import *
# import scipy as sp
# import latexify

$I_{D} = I_Sln^2[1+exp(\frac{V_G-V_{Th0}}{2nV_T})] = 2 \mu C_{ox}nV_T^2\frac{W}{L}ln^2[1+exp(\frac{V_G-V_{Th0}}{2nV_T})]$

$I_S = 689 nA, V_{Th0}: 387 mV, \kappa: 0.822$

VG = [0, 1]
Is: 727nA, VTh0: 400mV, kappa: 0.816

$I_S = 2 \mu C_{ox}nV_T^2\frac{W}{L} = \frac{2 \mu C_{ox}V_T^2\frac{W}{L}}{\kappa}$

<!-- $\frac{W}{L} = \frac{\kappa I_S}{\mu C_{ox}V_T^2}$ -->

$  \frac{2\mu C_{ox}V_T^2}{\kappa} = \frac{I_S}{W/L}$

$\mu C_{ox} = \frac{I_S \kappa}{2(W/L)V_T^2}$

In [315]:
W_L = 2         #W/L was 2u/1u for the reference transistors 
VT = 0.0258

Is_n = siString2Num("689n")
Vth0_n = siString2Num("387m")
k_n = 0.822
n_n = 1/k_n
uCox_n = (Is_n*k_n)/(2*2*VT**2)
u_n = 1.343e-2
lambdac_n = 0.04



Is_p = siString2Num("904.8n")
Vth0_p = siString2Num("371m")
k_p = 0.816
n_p = 1/k_p
uCox_p = (Is_p*k_p)/(2*2*VT**2)
u_p = 5.2e-3
lambdac_n = 0.02

## Specificatii
|Tip OTA |Gain |GBW | C_L| VDD | VSS | IQ |
|---|---|---|---|---|---|---|
FC NMOS |70dB | 40 MHz | 5pF | 1.65| -1.65| 100uA|

In [316]:
gbw = siString2Num("40M")
gain = 10**(70/20)              # 70dB to linear
C_L = siString2Num("5p")
VDD = 1.65
VSS = -1.65
IQ = siString2Num("100u")

$gm_1 = 2\pi GBW C_L$

In [317]:
gm1 = 2*math.pi * gbw * C_L
print(f"gm1,2 = {num2SiStringRounded(gm1)}S")

gm1,2 = 1.257mS


$gm = \frac{I_D}{nV_T}\cdot \frac{1-exp(-\sqrt{IC})}{\sqrt{IC}} = \frac{kI_D}{V_T}\cdot \frac{1-exp(-\sqrt{IC})}{\sqrt{IC}}$

$IC = \frac{I_D}{I_S} = \frac{I_D}{2 \mu C_{ox}nV_T^2\frac{W}{L}}$ = coeficientul de inversie

$I_D = \frac{gmV_T\sqrt{IC}}{\kappa[1-exp(-\sqrt{IC})]}$

In [318]:
# gm, Id, k, IC, VT = sp.symbols('gm Id k IC VT')
# eq = gm - k*Id/VT*(1-sp.exp(-IC))/sp.sqrt(IC)
# sol = sp.solve(eq, IC)

In [319]:
IC = 0.8
ID = (gm1*VT*np.sqrt(IC))/(k_n*(1-np.exp(-np.sqrt(IC))))
print(f"ID = {num2SiStringRounded(ID)}A")
IBiasDiffPair = 2*ID

ID = 59.676uA


$\frac{W}{L} = \frac{I_D}{2 \mu C_{ox}nV_T^2IC}$

In [320]:
def getDimensions(IC, ID, mostype):
    if mostype == "n":
        return (ID)/(2*uCox_n*n_n*VT**2*IC)
    elif mostype == "p":
        return (ID)/(2*uCox_p*n_p*VT**2*IC)
    else:
        raise ValueError("mostype has to be either \"n\" or \"p\"")

# W_L = (ID)/(2*uCox*n*VT**2*IC)
W_L1 = getDimensions(IC, ID, "n")
L = siString2Num("130n")
W = W_L1*L
# print(W)
print(f"W/L_1,2 = {num2SiStringRounded(W)}/{num2SiStringRounded(L)}")

W/L_1,2 = 28.149u/130.0n


$f_t = \frac{\mu V_T}{2\pi L^2}\cdot(\sqrt{1+IC}-1)$

In [321]:
ft = (u_n*VT)/(2*np.pi*L**2)*(np.sqrt(1+IC)-1)
print(f"ft = {num2SiStringRounded(ft)}Hz")
if ft < gbw:
    print(f"ft < gbw = {num2SiStringRounded(gbw)}Hz")
else:
    print(f"ft > gbw = {num2SiStringRounded(gbw)}Hz")

ft = 1.115GHz
ft > gbw = 40.0MHz


## M10, M11 Dimensioning
$I_{DS\_10,11} > I_{Bias\_DiffPair}$

In [322]:
I10_11 = 1.2*IBiasDiffPair
print(f"ID_10,11 = {num2SiStringRounded(I10_11)}")
IC = 20      # strong inversion

W_L_10_11 = getDimensions(IC, I10_11, "p")
L = siString2Num("1u")
W = W_L_10_11 * L
print(f"W/L_10,11 = {num2SiStringRounded(W)}/{num2SiStringRounded(L)}")

def getVDSat(IC):
    return VT*(np.sqrt(1+IC)-1 + 4)
VDSat = getVDSat(IC)
print(f"VDSat_10,11 = {num2SiStringRounded(VDSat)}")


ID_10,11 = 143.222u
W/L_10,11 = 15.829u/1.0u
VDSat_10,11 = 195.63m


In [323]:
ft = (u_p*VT)/(2*np.pi*L**2)*(np.sqrt(1+IC)-1)
print(f"ft = {num2SiStringRounded(ft)}Hz")
if ft < gbw:
    print(f"ft < gbw = {num2SiStringRounded(gbw)}Hz")
else:
    print(f"ft > gbw = {num2SiStringRounded(gbw)}Hz")

ft = 76.496MHz
ft > gbw = 40.0MHz


## M4, M5 Dimensioning

$I_{DS\_4,4} = I_{DS\_10,11} - I_{Bias\_DiffPair}/2$

In [324]:
I4_5 = I10_11 -IBiasDiffPair/2
print(f"IDS_4,5 = {num2SiStringRounded(I4_5)}")
IC = 20      # strong inversion
W_L_4_5 = getDimensions(IC, I10_11, "n")
L = siString2Num("1u")
W = W_L_4_5 * L
print(f"W/L_4,5 = {num2SiStringRounded(W)}/{num2SiStringRounded(L)}")
print(f"VDSat_4,5 = {num2SiStringRounded(getVDSat(IC))}")

IDS_4,5 = 83.546u
W/L_4,5 = 20.787u/1.0u
VDSat_4,5 = 195.63m


In [325]:
ft = (u_n*VT)/(2*np.pi*L**2)*(np.sqrt(1+IC)-1)
print(f"ft = {num2SiStringRounded(ft)}Hz")
if ft < gbw:
    print(f"ft < gbw = {num2SiStringRounded(gbw)}Hz")
else:
    print(f"ft > gbw = {num2SiStringRounded(gbw)}Hz")

ft = 197.566MHz
ft > gbw = 40.0MHz


## M6, M7 Dimensioning
$I_{DS\_6,7} = I_{DS\_10,11} - I_{Bias\_DiffPair}/2$

In [326]:
I6_7 = I10_11 - IBiasDiffPair/2
print(f"IDS_6,7 = {num2SiStringRounded(I6_7)}")
IC = 10      # strong inversion
W_L_6_7 = getDimensions(IC, I6_7, "n")
L = siString2Num("130n")
W = W_L_6_7 * L
print(f"W/L_6,7 = {num2SiStringRounded(W)}/{num2SiStringRounded(L)}")
print(f"VDSat_6,7 = {num2SiStringRounded(getVDSat(IC))}")


IDS_6,7 = 83.546u
W/L_6,7 = 3.153u/130.0n
VDSat_6,7 = 162.969m


In [327]:
ft = (u_n*VT)/(2*np.pi*L**2)*(np.sqrt(1+IC)-1)
print(f"ft = {num2SiStringRounded(ft)}Hz")
if ft < gbw:
    print(f"ft < gbw = {num2SiStringRounded(gbw)}Hz")
else:
    print(f"ft > gbw = {num2SiStringRounded(gbw)}Hz")

ft = 7.559GHz
ft > gbw = 40.0MHz


## M8, M9 Dimensioning
$I_{DS\_8,9} = I_{DS\_10,11} - I_{Bias\_DiffPair}/2$

In [328]:
I8_9 = I10_11 - IBiasDiffPair/2
print(f"IDS_8,9 = {num2SiStringRounded(I8_9)}")
IC = 10      # strong inversion
W_L_8_9 = getDimensions(IC, I8_9, "p")
L = siString2Num("130n")
W = W_L_8_9 * L
print(f"W/L_8,9 = {num2SiStringRounded(W)}/{num2SiStringRounded(L)}")
print(f"VDSat_6,7 = {num2SiStringRounded(getVDSat(IC))}")


IDS_8,9 = 83.546u
W/L_8,9 = 2.401u/130.0n
VDSat_6,7 = 162.969m


In [329]:
ft = (u_p*VT)/(2*np.pi*L**2)*(np.sqrt(1+IC)-1)
print(f"ft = {num2SiStringRounded(ft)}Hz")
if ft < gbw:
    print(f"ft < gbw = {num2SiStringRounded(gbw)}Hz")
else:
    print(f"ft > gbw = {num2SiStringRounded(gbw)}Hz")

ft = 2.927GHz
ft > gbw = 40.0MHz


# Do a design handbook for the folded cascode
# get design equations for the folded cascode; mainly find the small signal parameters: gm, rds, Cgs, Cgd, gmb??
# Write a comparative demonstration between a design following square law equations and one following the ekv model